In [82]:
import pandas as pd
import geopandas as gpd
from collections import Counter
import os

***

### Step 1: import all WFS layers into separate gdfs

**Importing faciliteter layers**

In [2]:
indk = gpd.read_file("../data/raw/wfs/facilit_faciliteter/indkoeb.gpkg").explode(index_parts=False)
info = gpd.read_file("../data/raw/wfs/facilit_faciliteter/infoservice.gpkg").explode(index_parts=False)
infosup = gpd.read_file("../data/raw/wfs/facilit_faciliteter/infoservice_suppl.gpkg").explode(index_parts=False)
over = gpd.read_file("../data/raw/wfs/facilit_faciliteter/overnatning.gpkg").explode(index_parts=False)
rast = gpd.read_file("../data/raw/wfs/facilit_faciliteter/rasteplads.gpkg").explode(index_parts=False)
rastsup = gpd.read_file("../data/raw/wfs/facilit_faciliteter/rasteplads_suppl.gpkg").explode(index_parts=False)

**Importing land_anvendelse layers**

In [3]:
byom = gpd.read_file("../data/raw/wfs/land_anvendelse/byomraade.gpkg").explode(index_parts=False)
dyrk = gpd.read_file("../data/raw/wfs/land_anvendelse/dyrket_areal.gpkg").explode(index_parts=False)
natu = gpd.read_file("../data/raw/wfs/land_anvendelse/naturareal.gpkg").explode(index_parts=False)
skov = gpd.read_file("../data/raw/wfs/land_anvendelse/skovinddeling.gpkg").explode(index_parts=False)
tekn = gpd.read_file("../data/raw/wfs/land_anvendelse/teknisk_areal.gpkg").explode(index_parts=False)

**Importing land_attraktioner layers**

In [4]:
beso = gpd.read_file("../data/raw/wfs/land_attraktioner/besoeg.gpkg").explode(index_parts=False)
fort = gpd.read_file("../data/raw/wfs/land_attraktioner/fortidsminder.gpkg").explode(index_parts=False)
land = gpd.read_file("../data/raw/wfs/land_attraktioner/landemaerker.gpkg").explode(index_parts=False)
udfl = gpd.read_file("../data/raw/wfs/land_attraktioner/udflugt.gpkg").explode(index_parts=False)

In [41]:
Counter(udfl["type"])

Counter({'Naturområde': 7,
         'Udsigtspunkt': 6,
         'Badestrand/-sted': 4,
         'Fugletårn/Udsigtstårn': 1})

**Importing landskabnatur layers** 

ATTENTION SOME LAYERS ARE MISSING!

In [5]:
besk = gpd.read_file("../data/raw/wfs/land_landskabnatur/beskyttet_natur.gpkg").explode(index_parts=False)

# missing:
# Fredninger, nationalparker mm.; Værdifulde landskaber mm

***

### Step 2: Combine gdfs into evaluation layers

we want to only keep geometries and types

In [71]:
def merge_gdfs(gdf_list):

    # make sure all gdfs are the same crs
    assert len(set([gdf.crs for gdf in gdf_list]))==1
    # make sure we have the same geometries everywhere
    assert len(set([t for gdf in gdf_list for t in gdf.type]))==1
    # concatenate with pandas
    gdf_main = pd.concat([gdf[["geometry", "type"]].copy() for gdf in gdf_list])    
    
    return gdf_main


**Nature**

In [72]:
nature = merge_gdfs(
    [
        natu[-natu["type"].isin(["Råstofomrade"])],
        skov,
        besk,
        # fred, 
        # vaer,    
    ]
    )
nature.head(3)

,geometry,type
0,"POLYGON ((674187.995 6132974.789, 674181.977 6...",Skov
1,"POLYGON ((674564.760 6133499.960, 674564.510 6...",Kratbevoksning
2,"POLYGON ((674543.590 6133496.290, 674543.490 6...",Kratbevoksning


**Culture**

In [73]:
# CULTURE
culture = merge_gdfs(
    [
        byom[byom["type"]=="Bykerne"],
        # vaer filtered by "kulturhistorisk..." and "kulturmiljø"
    ]
)
culture.head(3)

,geometry,type
0,"POLYGON ((688000.060 6134930.550, 688005.050 6...",Bykerne


**Sommerhus**

In [75]:
# SOMMERHUS
sommerhus = merge_gdfs(
    [
        byom[byom["type"].isin(["Sommerhusområde", "Sommerhusområde skov"])]
    ]
)
sommerhus.head(3)

,geometry,type
637,"POLYGON ((699967.057 6121418.232, 699949.159 6...",Sommerhusområde
638,"POLYGON ((695048.016 6118988.105, 695045.802 6...",Sommerhusområde
639,"POLYGON ((701900.987 6123607.918, 701856.971 6...",Sommerhusområde


**Agriculture**

Q: add some more to this layer?

In [76]:
agriculture = dyrk.copy() # agriculture!! (separate layer)
agriculture.head(3)

,tmp_id,gruppe,type,kilde,geometry
0,9584,Dyrkede arealer,Dyrkede afgrøder på marker,LBST,"POLYGON ((682785.132 6133401.399, 682784.586 6..."
1,9585,Dyrkede arealer,Dyrkede afgrøder på marker,LBST,"POLYGON ((682631.161 6133417.782, 682631.441 6..."
2,9586,Dyrkede arealer,Dyrkede afgrøder på marker,LBST,"POLYGON ((682629.405 6133418.333, 682629.373 6..."


**Bad**

In [77]:
bad = merge_gdfs(
    [
        byom[byom["type"].isin(["Erhverv", "Høj bebyggelse"])],
        tekn[-tekn["type"].isin(["Sportsanlæg"])]
    ]
)
bad.head(3)


,geometry,type
1,"POLYGON ((687678.460 6135211.880, 687681.560 6...",Høj bebyggelse
2,"POLYGON ((687478.210 6136000.000, 687482.040 6...",Høj bebyggelse
3,"POLYGON ((688000.020 6135481.940, 688009.110 6...",Høj bebyggelse


**POIs**

In [79]:
# merge beso, fort, udfl, land; just keep track of what type they are (variation is interesting!)
# in max 1km distance 
pois = merge_gdfs(
    [
        beso,
        fort,
        udfl,
        land
    ]
)
print(Counter(pois["type"]))
pois.head(3)

Counter({'Kirke': 29, 'Herregård': 18, 'Vold eller voldsted': 11, 'Gravhøj': 8, 'Naturområde': 7, 'Udsigtspunkt': 6, 'Museum': 5, 'Dysse': 4, 'Badestrand/-sted': 4, 'Forlystelsespark': 3, 'Anden seværdighed': 3, 'Fortidsminde/ruin': 3, 'Vejrmølle': 3, 'Aboret, blomsterpark o.l.': 1, 'Langdysse': 1, 'Fugletårn/Udsigtstårn': 1})


,geometry,type
0,POINT (678747.300 6130234.410),Museum
1,POINT (698473.362 6127698.601),Museum
2,POINT (688007.560 6135779.670),Museum


**Facilities**

In [80]:
# in max 100 m
# all info except for turistkontor ()
# all rasteplads
facilities = merge_gdfs(
    [
        info[-info["type"].isin(["turistkontor"])],
        infosup,
        rast,
        rastsup
    ]
)
facilities.head()

,geometry,type
0,POINT (688082.831 6135146.387),Toilet
1,POINT (687894.311 6135489.899),Toilet
2,POINT (697075.000 6118030.000),Toilet
3,POINT (687971.710 6135409.924),Cykelpumpestation/-servicepost
4,POINT (688043.233 6135050.839),Vandpost


**Service**

In [81]:
# in max 500 m
# all indk; over;
# turistkontor (from info)
service = merge_gdfs(
    [
        indk,
        over,
        info[info["type"].isin(["turistkontor"])]
    ]
)
service.head(3)

,geometry,type
0,POINT (688212.710 6140487.520),supermarked
1,POINT (688212.710 6140487.520),supermarked
2,POINT (681137.720 6127194.910),bager


***

## Step 3: Save into files

In [83]:
os.makedirs("../data/processed/eval/", exist_ok=True)

In [85]:
agriculture.to_file("../data/processed/eval/agriculture.gpkg", index=False)
bad.to_file("../data/processed/eval/bad.gpkg", index=False)
culture.to_file("../data/processed/eval/culture.gpkg", index=False)
facilities.to_file("../data/processed/eval/facilities.gpkg", index=False)
nature.to_file("../data/processed/eval/nature.gpkg", index=False)
pois.to_file("../data/processed/eval/pois.gpkg", index=False)
service.to_file("../data/processed/eval/service.gpkg", index=False)
sommerhus.to_file("../data/processed/eval/sommerhus.gpkg", index=False)